<a href="https://colab.research.google.com/github/arjunaju123/Graph-RAG-using-Groq-Llama3-and-Neo4j/blob/master/Graph_RAG_using_Groq_Llama3_and_Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Realtime Powerful RAG Pipeline using Neo4j(Knowledge Graph Db) ,Langchain, Groq-Llama3

The use of retrieval-augmented generation (RAG) to retrieve relevant information from an external knowledge source enables large language models (LLMs) to answer questions over private and/or previously unseen document collections. However, RAG fails on global questions directed at an entire text corpus, such as “What are the main themes in the dataset?”, since this is inherently a queryfocused summarization (QFS) task, rather than an explicit retrieval task. Prior QFS methods, meanwhile, fail to scale to the quantities of text indexed by typical RAGsystems. Tocombine the strengths of these contrasting methods, we propose a Graph RAGapproach to question answering over private text corpora that scales with both the generality of user questions and the quantity of source text to be indexed. Our approach uses an LLM to build a graph-based text index in two stages: f irst to derive an entity knowledge graph from the source documents, then to pregenerate community summaries for all groups of closely-related entities. Given a question, each community summary is used to generate a partial response, before all partial responses are again summarized in a final response to the user. For a class of global sensemaking questions over datasets in the 1 million token range, we show that Graph RAG leads to substantial improvements over a na¨ ıve RAG baseline for both the comprehensiveness and diversity of generated answers.

# https://arxiv.org/abs/2404.16130

# https://docs.google.com/document/d/1vpKUtWJtc03G460u3c6OtTUTtadAUGtX6B94BahN57M/edit

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 

In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [4]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [5]:
from typing import Tuple, List, Optional

In [6]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [7]:
from langchain_core.runnables import ConfigurableField

In [8]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [9]:
import os

In [10]:

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass


In [11]:
from langchain_community.vectorstores import Neo4jVector

In [14]:
NEO4J_URI='neo4j+s://b082fd26.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD='bzuQFTxhM38FwzCIL2GMA-GOwA0RbqcMpeN-EOTabvM'
# AURA_INSTANCEID=b082fd26
# AURA_INSTANCENAME=Instance01

In [15]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [16]:
from langchain_community.graphs import Neo4jGraph

In [18]:
graph = Neo4jGraph()

In [19]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [20]:
len(raw_documents)

23

In [21]:
raw_documents[:3]

[Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was i

In [22]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [23]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.9 MB/s eta 0:00:00


In [25]:
from groq import Groq

from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')

#os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or getpass.getpass("Enter your Groq API key: ")

groq_client = Groq(api_key=GROQ_API_KEY)

In [26]:
# from langchain_openai import ChatOpenAI
# llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [27]:
# llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7d36c3a41ae0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7d36c3a431f0>, model_name='gpt-3.5-turbo-0125', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

In [34]:
%pip install -qU langchain-groq

In [54]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key=GROQ_API_KEY # Optional if not set as an environment variable
)

In [55]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7d36c1c6cbe0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7d36c1c6c220>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [30]:
# from langchain_experimental.graph_transformers import LLMGraphTransformer
# llm_transformer = LLMGraphTransformer(llm=llm)

In [31]:
# llm_transformer

In [38]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [39]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [40]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person'), Node(id='Henry Viii', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Edward Vi', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Mary', type='Person'), Node(id='William Cecil', type='Person'), Node(id='James Vi Of Scotland', type='Person'), Node(id='Mary, Queen Of Scots', type='Person'), Node(id='Sir Francis Walsingham', type='Person'), Node(id='England', type='Location'), Node(id='Ireland', type='Location'), Node(id='Scotland', type='Location'), Node(id='France', type='Location'), Node(id='Spain', type='Location'), Node(id='Netherlands', type='Location')], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Henry Viii', type='Person'), type='PARENT'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='Anne Boleyn', type='Person'), type='PARENT'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='England'

In [41]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [42]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [43]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [44]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [45]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [46]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [47]:
from typing import Tuple, List, Optional

In [48]:
from langchain_community.vectorstores import Neo4jVector

In [49]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [80]:
OpenAIEmbeddings()

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7d36c0f3df30>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7d36c0fc3190>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

Using an embedding model trained or fine-tuned for the same domain as the LLM can significantly improve a RAG system. However, finding or training such an embedding model is often a difficult task as in-domain data are usually scarce.

how to turn an LLM into a text embedding model using LLM2Vec. We will see how to do it with Llama 3 to create a RAG system that doesn’t need any other models than Llama 3.

### https://kaitchup.substack.com/p/turn-llama-3-into-an-embedding-model

In [81]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [51]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

In [52]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [53]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [56]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [57]:
entity_chain.invoke({"question": "Where was Amelia Earhart born?"}).names

['Amelia Earhart']

In [58]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [59]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [60]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [61]:
print(structured_retriever("Who is Elizabeth I?"))

Elizabeth I - PARENT -> Henry Viii
Elizabeth I - PARENT -> Anne Boleyn
Elizabeth I - MONARCH -> England
Elizabeth I - MONARCH -> Ireland
William Cecil - ADVISER -> Elizabeth I
Sir Francis Walsingham - MINISTER -> Elizabeth I
Queen Elizabeth I - FATHER -> Henry Viii
Queen Elizabeth I - OFFICIAL_LIMNER -> Nicholas Hilliard
Queen Elizabeth I - SERJEANT_PAINTER -> George Gower


In [62]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [63]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [64]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [65]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [73]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(temperature=0,model="llama3-70b-8192",api_key=GROQ_API_KEY)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [74]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [75]:
prompt = ChatPromptTemplate.from_template(template)

In [76]:

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [77]:
chain.invoke({"question": "Which house did Elizabeth I belong to?"})

Search query: Which house did Elizabeth I belong to?


'Elizabeth I belonged to the House of Tudor.'

In [78]:

chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


'Elizabeth I was born on September 7, 1533.'